# DRUGBANK VERSION #

In [1]:
# Import everything needed
from matplotlib.patches import Patch
from scipy import stats
from sklearn.metrics import r2_score
from statsmodels.stats.multitest import multipletests
from toxicity_ranking import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scikit_posthocs as sp

In [2]:
drug_syntox_df = pd.read_csv('data_processed/drugbank_syntox_known.csv')
print('DrugSynergyToxicity shape: ', drug_syntox_df.shape)
print(drug_syntox_df.head())

DrugSynergyToxicity shape:  (88219, 8)
     drug_row     drug_col cell_line_name  synergy_zip  synergy_loewe  \
0  romidepsin  carfilzomib           EW-8     3.397628      -1.793540   
1  romidepsin  carfilzomib           EW-8    16.630494       0.941715   
2  romidepsin  carfilzomib           EW-8    14.782992      26.427238   
3   tamoxifen  niclosamide          TC-32    -1.251659     -15.929260   
4   dasatinib  niclosamide          TC-32     1.292635     -31.445789   

   synergy_bliss  synergy_hsa toxicity_category  
0       2.087050     1.989756             Major  
1      15.313094     5.341596             Major  
2      26.653928    29.164734             Major  
3      -1.513906    -2.698921             Major  
4       4.202408     1.510223             Major  


In [3]:
# Create histogram for each synergy score: bliss, loewe, hsa, zip
drug_syntox_df.hist(column='synergy_bliss', bins=100)
plt.xlabel('Bliss synergy score')
plt.ylabel('Frequency')
plt.title('Bliss synergy score distribution in DrugComb-DrugBank intersection')
plt.savefig('results/synergy_score_distrib/bliss_drugbank.png')
plt.close()

drug_syntox_df.hist(column='synergy_loewe', bins=100)
plt.xlabel('Loewe synergy score')
plt.ylabel('Frequency')
plt.title('Loewe synergy score distribution in DrugComb-DrugBank intersection')
plt.savefig('results/synergy_score_distrib/loewe_drugbank.png')
plt.close()

drug_syntox_df.hist(column='synergy_hsa', bins=100)
plt.xlabel('HSA synergy score')
plt.ylabel('Frequency')
plt.title('HSA synergy score distribution in DrugComb-DrugBank intersection')
plt.savefig('results/synergy_score_distrib/hsa_drugbank.png')
plt.close()

drug_syntox_df.hist(column='synergy_zip', bins=100)
plt.xlabel('ZIP synergy score')
plt.ylabel('Frequency')
plt.title('ZIP synergy score distribution in DrugComb-DrugBank intersection')
plt.savefig('results/synergy_score_distrib/zip_drugbank.png')
plt.close()


# What's the max and minimum for each synergy score?
print('Max Bliss synergy:', drug_syntox_df['synergy_bliss'].max())
print('Min Bliss synergy:', drug_syntox_df['synergy_bliss'].min())
print(stats.normaltest(drug_syntox_df['synergy_bliss'])) # if smaller than 0.05 then not normal
print('Max HSA synergy:', drug_syntox_df['synergy_hsa'].max())
print('Min HSA synergy:', drug_syntox_df['synergy_hsa'].min())
print(stats.normaltest(drug_syntox_df['synergy_hsa'])) # if smaller than 0.05 then not normal
print('Max Loewe synergy:', drug_syntox_df['synergy_loewe'].max())
print('Min Loewe synergy:', drug_syntox_df['synergy_loewe'].min())
print(stats.normaltest(drug_syntox_df['synergy_loewe'])) # if smaller than 0.05 then not normal
print('Max ZIP synergy:', drug_syntox_df['synergy_zip'].max())
print('Min ZIP synergy:', drug_syntox_df['synergy_zip'].min())
print(stats.normaltest(drug_syntox_df['synergy_zip'])) # if smaller than 0.05 then not normal


Max Bliss synergy: 272.677271202622
Min Bliss synergy: -94.5613368419114
NormaltestResult(statistic=44646.05683022014, pvalue=0.0)
Max HSA synergy: 106.317660951751
Min HSA synergy: -113.031857050116
NormaltestResult(statistic=21555.41862614038, pvalue=0.0)
Max Loewe synergy: 94.7383399889338
Min Loewe synergy: -113.552208967779
NormaltestResult(statistic=19885.56920820311, pvalue=0.0)
Max ZIP synergy: 220.137358460667
Min ZIP synergy: -90.125490222854
NormaltestResult(statistic=29814.171702769258, pvalue=0.0)


In [4]:
# Create histogram for synergy_bliss for only Major toxicity category in drug_syntox_df
drug_syntox_major = drug_syntox_df[drug_syntox_df['toxicity_category'] == 'Major']

print('Synergy Bliss Median for Major Toxicity:', drug_syntox_major['synergy_bliss'].median())
print('Synergy Bliss Mean for Major Toxicity:', drug_syntox_major['synergy_bliss'].mean())
print('Synergy Bliss Variance for Major Toxicity:', drug_syntox_major['synergy_bliss'].var())
print('Synergy Bliss Standard Deviation for Major Toxicity:', drug_syntox_major['synergy_bliss'].std())

drug_syntox_major['synergy_bliss'].hist(bins=100)
plt.xlabel('Synergy Bliss')
plt.ylabel('Frequency')
plt.title('Synergy Bliss Distribution for Major Toxicity')
plt.savefig('results/synergy_score_tox_categ_distrib/bliss_major_drugbank.png')
plt.close()

# Create histogram for synergy_bliss for only Moderate toxicity category in drug_syntox_df
drug_syntox_moderate = drug_syntox_df[drug_syntox_df['toxicity_category'] == 'Moderate']

print('Synergy Bliss Median for Moderate Toxicity:', drug_syntox_moderate['synergy_bliss'].median())
print('Synergy Bliss Mean for Moderate Toxicity:', drug_syntox_moderate['synergy_bliss'].mean())
print('Synergy Bliss Variance for Moderate Toxicity:', drug_syntox_moderate['synergy_bliss'].var())
print('Synergy Bliss Standard Deviation for Moderate Toxicity:', drug_syntox_moderate['synergy_bliss'].std())

drug_syntox_moderate['synergy_bliss'].hist(bins=100)
plt.xlabel('Synergy Bliss')
plt.ylabel('Frequency')
plt.title('Synergy Bliss Distribution for Moderate Toxicity')
plt.savefig('results/synergy_score_tox_categ_distrib/bliss_moderate_drugbank.png')
plt.close()

# Create histogram for synergy_bliss for only Minor toxicity category in drug_syntox_df
drug_syntox_minor = drug_syntox_df[drug_syntox_df['toxicity_category'] == 'Minor']

print('Synergy Bliss Median for Minor Toxicity:', drug_syntox_minor['synergy_bliss'].median())
print('Synergy Bliss Mean for Minor Toxicity:', drug_syntox_minor['synergy_bliss'].mean())
print('Synergy Bliss Variance for Minor Toxicity:', drug_syntox_minor['synergy_bliss'].var())
print('Synergy Bliss Standard Deviation for Minor Toxicity:', drug_syntox_minor['synergy_bliss'].std())

drug_syntox_minor['synergy_bliss'].hist(bins=100)
plt.xlabel('Synergy Bliss')
plt.ylabel('Frequency')
plt.title('Synergy Bliss Distribution for Minor Toxicity')
plt.savefig('results/synergy_score_tox_categ_distrib/bliss_minor_drugbank.png')
plt.close()

# Create histogram for synergy_HSA for only Major toxicity category in drug_syntox_df
print('Synergy HSA Median for Major Toxicity:', drug_syntox_major['synergy_hsa'].median())
print('Synergy HSA Mean for Major Toxicity:', drug_syntox_major['synergy_hsa'].mean())
print('Synergy HSA Variance for Major Toxicity:', drug_syntox_major['synergy_hsa'].var())
print('Synergy HSA Standard Deviation for Major Toxicity:', drug_syntox_major['synergy_hsa'].std())

drug_syntox_major['synergy_hsa'].hist(bins=100)
plt.xlabel('Synergy HSA')
plt.ylabel('Frequency')
plt.title('Synergy HSA Distribution for Major Toxicity')
plt.savefig('results/synergy_score_tox_categ_distrib/hsa_major_drugbank.png')
plt.close()

# Create histogram for synergy_HSA for only Moderate toxicity category in drug_syntox_df
print('Synergy HSA Median for Moderate Toxicity:', drug_syntox_moderate['synergy_hsa'].median())
print('Synergy HSA Mean for Moderate Toxicity:', drug_syntox_moderate['synergy_hsa'].mean())
print('Synergy HSA Variance for Moderate Toxicity:', drug_syntox_moderate['synergy_hsa'].var())
print('Synergy HSA Standard Deviation for Moderate Toxicity:', drug_syntox_moderate['synergy_hsa'].std())

drug_syntox_moderate['synergy_hsa'].hist(bins=100)
plt.xlabel('Synergy HSA')
plt.ylabel('Frequency')
plt.title('Synergy HSA Distribution for Moderate Toxicity')
plt.savefig('results/synergy_score_tox_categ_distrib/hsa_moderate_drugbank.png')
plt.close()

# Create histogram for synergy_HSA for only Minor toxicity category in drug_syntox_df
print('Synergy HSA Median for Minor Toxicity:', drug_syntox_minor['synergy_hsa'].median())
print('Synergy HSA Mean for Minor Toxicity:', drug_syntox_minor['synergy_hsa'].mean())
print('Synergy HSA Variance for Minor Toxicity:', drug_syntox_minor['synergy_hsa'].var())
print('Synergy HSA Standard Deviation for Minor Toxicity:', drug_syntox_minor['synergy_hsa'].std())

drug_syntox_minor['synergy_hsa'].hist(bins=100)
plt.xlabel('Synergy HSA')
plt.ylabel('Frequency')
plt.title('Synergy HSA Distribution for Minor Toxicity')
plt.savefig('results/synergy_score_tox_categ_distrib/hsa_minor_drugbank.png')
plt.close()

# Create histogram for synergy_zip for only Major toxicity category in drug_syntox_df
print('Synergy ZIP Median for Major Toxicity:', drug_syntox_major['synergy_zip'].median())
print('Synergy ZIP Mean for Major Toxicity:', drug_syntox_major['synergy_zip'].mean())
print('Synergy ZIP Variance for Major Toxicity:', drug_syntox_major['synergy_zip'].var())
print('Synergy ZIP Standard Deviation for Major Toxicity:', drug_syntox_major['synergy_zip'].std())

drug_syntox_major['synergy_zip'].hist(bins=100)
plt.xlabel('Synergy ZIP')
plt.ylabel('Frequency')
plt.title('Synergy ZIP Distribution for Major Toxicity')
plt.savefig('results/synergy_score_tox_categ_distrib/zip_major_drugbank.png')
plt.close()

# Create histogram for synergy_zip for only Moderate toxicity category in drug_syntox_df
print('Synergy ZIP Median for Moderate Toxicity:', drug_syntox_moderate['synergy_zip'].median())
print('Synergy ZIP Mean for Moderate Toxicity:', drug_syntox_moderate['synergy_zip'].mean())
print('Synergy ZIP Variance for Moderate Toxicity:', drug_syntox_moderate['synergy_zip'].var())
print('Synergy ZIP Standard Deviation for Moderate Toxicity:', drug_syntox_moderate['synergy_zip'].std())

drug_syntox_moderate['synergy_zip'].hist(bins=100)
plt.xlabel('Synergy ZIP')
plt.ylabel('Frequency')
plt.title('Synergy ZIP Distribution for Moderate Toxicity')
plt.savefig('results/synergy_score_tox_categ_distrib/zip_moderate_drugbank.png')
plt.close()

# Create histogram for synergy_zip for only Minor toxicity category in drug_syntox_df
print('Synergy ZIP Median for Minor Toxicity:', drug_syntox_minor['synergy_zip'].median())
print('Synergy ZIP Mean for Minor Toxicity:', drug_syntox_minor['synergy_zip'].mean())
print('Synergy ZIP Variance for Minor Toxicity:', drug_syntox_minor['synergy_zip'].var())
print('Synergy ZIP Standard Deviation for Minor Toxicity:', drug_syntox_minor['synergy_zip'].std())

drug_syntox_minor['synergy_zip'].hist(bins=100)
plt.xlabel('Synergy ZIP')
plt.ylabel('Frequency')
plt.title('Synergy ZIP Distribution for Minor Toxicity')
plt.savefig('results/synergy_score_tox_categ_distrib/zip_minor_drugbank.png')
plt.close()

# Create histogram for synergy_loewe for only Major toxicity category in drug_syntox_df
print('Synergy Loewe Median for Major Toxicity:', drug_syntox_major['synergy_loewe'].median())
print('Synergy Loewe Mean for Major Toxicity:', drug_syntox_major['synergy_loewe'].mean())
print('Synergy Loewe Variance for Major Toxicity:', drug_syntox_major['synergy_loewe'].var())
print('Synergy Loewe Standard Deviation for Major Toxicity:', drug_syntox_major['synergy_loewe'].std())

drug_syntox_major['synergy_loewe'].hist(bins=100)
plt.xlabel('Synergy Loewe')
plt.ylabel('Frequency')
plt.title('Synergy Loewe Distribution for Major Toxicity')
plt.savefig('results/synergy_score_tox_categ_distrib/loewe_major_drugbank.png')
plt.close()

# Create histogram for synergy_loewe for only Moderate toxicity category in drug_syntox_df
print('Synergy Loewe Median for Moderate Toxicity:', drug_syntox_moderate['synergy_loewe'].median())
print('Synergy Loewe Mean for Moderate Toxicity:', drug_syntox_moderate['synergy_loewe'].mean())
print('Synergy Loewe Variance for Moderate Toxicity:', drug_syntox_moderate['synergy_loewe'].var())
print('Synergy Loewe Standard Deviation for Moderate Toxicity:', drug_syntox_moderate['synergy_loewe'].std())

drug_syntox_moderate['synergy_loewe'].hist(bins=100)
plt.xlabel('Synergy Loewe')
plt.ylabel('Frequency')
plt.title('Synergy Loewe Distribution for Moderate Toxicity')
plt.savefig('results/synergy_score_tox_categ_distrib/loewe_moderate_drugbank.png')
plt.close()

# Create histogram for synergy_loewe for only Minor toxicity category in drug_syntox_df
print('Synergy Loewe Median for Minor Toxicity:', drug_syntox_minor['synergy_loewe'].median())
print('Synergy Loewe Mean for Minor Toxicity:', drug_syntox_minor['synergy_loewe'].mean())
print('Synergy Loewe Variance for Minor Toxicity:', drug_syntox_minor['synergy_loewe'].var())
print('Synergy Loewe Standard Deviation for Minor Toxicity:', drug_syntox_minor['synergy_loewe'].std())

drug_syntox_minor['synergy_loewe'].hist(bins=100)
plt.xlabel('Synergy Loewe')
plt.ylabel('Frequency')
plt.title('Synergy Loewe Distribution for Minor Toxicity')
plt.savefig('results/synergy_score_tox_categ_distrib/loewe_minor_drugbank.png')
plt.close()

Synergy Bliss Median for Major Toxicity: 0.1730165745659295
Synergy Bliss Mean for Major Toxicity: 0.4018953732211083
Synergy Bliss Variance for Major Toxicity: 183.4887779323172
Synergy Bliss Standard Deviation for Major Toxicity: 13.545802963734456
Synergy Bliss Median for Moderate Toxicity: 0.005043083466700651
Synergy Bliss Mean for Moderate Toxicity: -1.1852317403563772
Synergy Bliss Variance for Moderate Toxicity: 272.7014646375549
Synergy Bliss Standard Deviation for Moderate Toxicity: 16.513675079689406
Synergy Bliss Median for Minor Toxicity: -1.9438237106732
Synergy Bliss Mean for Minor Toxicity: -5.348871497127659
Synergy Bliss Variance for Minor Toxicity: 264.5542949509054
Synergy Bliss Standard Deviation for Minor Toxicity: 16.265125113287798
Synergy HSA Median for Major Toxicity: -0.214134592566313
Synergy HSA Mean for Major Toxicity: -0.7033577936681688
Synergy HSA Variance for Major Toxicity: 95.86602271841281
Synergy HSA Standard Deviation for Major Toxicity: 9.7911195

- Kruskal Wallis Test
- Dunn Posthoc Test with Bonferroni Correction
- Jonckheere Terpestra Test
- ANOVA
- T Test with Bonferroni Correction

In [5]:
# Are there significant differences in the distributions of synergy scores between Major, Moderate, and Minor toxicity categories?
# Filter drug_syntox_df for only Major, Moderate, and Minor toxicity categories
drug_syntox_df = drug_syntox_df[drug_syntox_df['toxicity_category'].isin(['Major', 'Moderate', 'Minor'])]
colors = ['#20965D', '#FFBC42', '#D81159']
tox_order = ['Minor', 'Moderate', 'Major']
color_dict = dict(zip(tox_order, colors))
legend_elements = [
    Patch(facecolor=color_dict[cat], label = cat) for cat in tox_order
]

#Bliss
major_bliss = drug_syntox_df[drug_syntox_df['toxicity_category'] == 'Major']['synergy_bliss']
moderate_bliss = drug_syntox_df[drug_syntox_df['toxicity_category'] == 'Moderate']['synergy_bliss']
minor_bliss = drug_syntox_df[drug_syntox_df['toxicity_category'] == 'Minor']['synergy_bliss']

h_statistic_bliss, p_value_bliss = stats.kruskal(major_bliss, moderate_bliss, minor_bliss)
print(f'Kruskal-Wallis H statistic for Bliss synergy: {h_statistic_bliss}')
print(f'P-value for Bliss synergy: {p_value_bliss}')

dunn_bliss = sp.posthoc_dunn(drug_syntox_df, val_col='synergy_bliss', group_col='toxicity_category', p_adjust='bonferroni')
print('Dunn post-hoc test for Bliss synergy:')
print(dunn_bliss)

bliss_major_samples = major_bliss.values.tolist()
bliss_moderate_samples = moderate_bliss.values.tolist()
bliss_minor_samples = minor_bliss.values.tolist()
bliss_increasing_tox_samples = [bliss_minor_samples, bliss_moderate_samples, bliss_major_samples]
jt_incr_bliss = jonckheere_terpestra_test(bliss_increasing_tox_samples)
print("Increasing toxicity for Bliss: ", jt_incr_bliss )
bliss_decreasing_tox_samples = [ bliss_major_samples, bliss_moderate_samples, bliss_minor_samples]
jt_decr_bliss = jonckheere_terpestra_test(bliss_decreasing_tox_samples)
print("Decreasing toxicity for Bliss: ", jt_decr_bliss)

# Run an ANOVA on the Bliss synergy scores for Major, Moderate, and Minor toxicity categoreis
f_statistic_anova_bliss_tox, p_value_anova_bliss_tox = stats.f_oneway(major_bliss, moderate_bliss, minor_bliss)
print(f'ANOVA F-statistic for bliss: {f_statistic_anova_bliss_tox}')
print(f'P-value for bliss: {p_value_anova_bliss_tox}')

# Run a T test on bliss between Major/Minor, Major/Moderate, and Moderate/Minor toxicity categories
major_minor_bliss = stats.ttest_ind(major_bliss, minor_bliss)
major_moderate_bliss = stats.ttest_ind(major_bliss, moderate_bliss)
moderate_minor_bliss = stats.ttest_ind(moderate_bliss, minor_bliss)

# Run bonferroni correction on the p-values
ttest_p_values_bliss = [major_minor_bliss[1], major_moderate_bliss[1], moderate_minor_bliss[1]]
ttest_p_values_corrected_bliss = multipletests(ttest_p_values_bliss, method='bonferroni')
print(f'Major/Minor T-test: {major_minor_bliss[1]} Corrected: {ttest_p_values_corrected_bliss[1][0]}')
print(f'Major/Moderate T-test: {major_moderate_bliss[1]} Corrected: {ttest_p_values_corrected_bliss[1][1]}')
print(f'Moderate/Minor T-test: {moderate_minor_bliss[1]} Corrected: {ttest_p_values_corrected_bliss[1][2]}')

ax = sns.violinplot(data=drug_syntox_df, x='toxicity_category', y='synergy_bliss', palette=color_dict, hue='toxicity_category', order=tox_order)
ax.set_xlabel('')
ax.set_ylabel('Bliss Synergy Score')
plt.legend(handles=legend_elements, bbox_to_anchor=(0.5,-0.2), loc='center', ncol=3)
plt.tight_layout()
plt.savefig('results/figure2/bliss_v_toxcats_drugbank_violin.png', dpi=700)
plt.tight_layout()
plt.close()

ax = sns.stripplot(data=drug_syntox_df, x='toxicity_category', y='synergy_bliss', palette=color_dict, hue='toxicity_category', order=tox_order)
ax.set_xlabel('')
ax.set_ylabel('Bliss Synergy Score')
plt.legend(handles=legend_elements, bbox_to_anchor=(0.5,-0.2), loc='center', ncol=3)
plt.tight_layout()
plt.savefig('results/figure2/bliss_v_toxcats_drugbank_strip.png', dpi=700)
plt.tight_layout()
plt.close()

#HSA
major_hsa = drug_syntox_df[drug_syntox_df['toxicity_category'] == 'Major']['synergy_hsa']
moderate_hsa = drug_syntox_df[drug_syntox_df['toxicity_category'] == 'Moderate']['synergy_hsa']
minor_hsa = drug_syntox_df[drug_syntox_df['toxicity_category'] == 'Minor']['synergy_hsa']

h_statistic_hsa, p_value_hsa = stats.kruskal(major_hsa, moderate_hsa, minor_hsa)
print(f'Kruskal-Wallis H statistic for HSA synergy: {h_statistic_hsa}')
print(f'P-value for HSA synergy: {p_value_hsa}')

dunn_hsa = sp.posthoc_dunn(drug_syntox_df, val_col='synergy_hsa', group_col='toxicity_category', p_adjust='bonferroni')
print('Dunn post-hoc test for HSA synergy:')
print(dunn_hsa)

hsa_major_samples = major_hsa.values.tolist()
hsa_moderate_samples = moderate_hsa.values.tolist()
hsa_minor_samples = minor_hsa.values.tolist()
hsa_increasing_tox_samples = [hsa_minor_samples, hsa_moderate_samples, hsa_major_samples]
jt_incr_hsa = jonckheere_terpestra_test(hsa_increasing_tox_samples)
print("Increasing toxicity for HSA: ", jt_incr_hsa )
hsa_decreasing_tox_samples = [ hsa_major_samples, hsa_moderate_samples, hsa_minor_samples]
jt_decr_hsa = jonckheere_terpestra_test(hsa_decreasing_tox_samples)
print("Decreasing toxicity for HSA: ", jt_decr_hsa)

# Run an ANOVA on the HSA synergy scores for Major, Moderate, and Minor toxicity categoreis
f_statistic_anova_hsa_tox, p_value_anova_hsa_tox = stats.f_oneway(major_hsa, moderate_hsa, minor_hsa)
print(f'ANOVA F-statistic for HSA: {f_statistic_anova_hsa_tox}')
print(f'P-value for HSA: {p_value_anova_hsa_tox}')

# Run a T test on HSA between Major/Minor, Major/Moderate, and Moderate/Minor toxicity categories
major_minor_hsa = stats.ttest_ind(major_hsa, minor_hsa)
major_moderate_hsa = stats.ttest_ind(major_hsa, moderate_hsa)
moderate_minor_hsa = stats.ttest_ind(moderate_hsa, minor_hsa)

# Run bonferroni correction on the p-values
ttest_p_values_hsa = [major_minor_hsa[1], major_moderate_hsa[1], moderate_minor_hsa[1]]
ttest_p_values_corrected_hsa = multipletests(ttest_p_values_hsa, method='bonferroni')
print(f'Major/Minor T-test: {major_minor_hsa[1]} Corrected: {ttest_p_values_corrected_hsa[1][0]}')
print(f'Major/Moderate T-test: {major_moderate_hsa[1]} Corrected: {ttest_p_values_corrected_hsa[1][1]}')
print(f'Moderate/Minor T-test: {moderate_minor_hsa[1]} Corrected: {ttest_p_values_corrected_hsa[1][2]}')

ax = sns.violinplot(data=drug_syntox_df, x='toxicity_category', y='synergy_hsa', palette=color_dict, hue='toxicity_category', order=tox_order)
ax.set_xlabel('')
ax.set_ylabel('HSA Synergy Score')
plt.legend(handles=legend_elements, bbox_to_anchor=(0.5,-0.2), loc='center', ncol=3)
plt.tight_layout()
plt.savefig('results/figure2/hsa_v_toxcats_drugbank.png', dpi=700)
plt.close()

ax = sns.stripplot(data=drug_syntox_df, x='toxicity_category', y='synergy_hsa', palette=color_dict, hue='toxicity_category', order=tox_order)
ax.set_xlabel('')
ax.set_ylabel('HSA Synergy Score')
plt.legend(handles=legend_elements, bbox_to_anchor=(0.5,-0.2), loc='center', ncol=3)
plt.tight_layout()
plt.savefig('results/figure2/hsa_v_toxcats_drugbank_strip.png', dpi=700)
plt.tight_layout()
plt.close()

#ZIP
major_zip = drug_syntox_df[drug_syntox_df['toxicity_category'] == 'Major']['synergy_zip']
moderate_zip = drug_syntox_df[drug_syntox_df['toxicity_category'] == 'Moderate']['synergy_zip']
minor_zip = drug_syntox_df[drug_syntox_df['toxicity_category'] == 'Minor']['synergy_zip']

h_statistic_zip, p_value_zip = stats.kruskal(major_zip, moderate_zip, minor_zip)
print(f'Kruskal-Wallis H statistic for ZIP synergy: {h_statistic_zip}')
print(f'P-value for ZIP synergy: {p_value_zip}')

dunn_zip = sp.posthoc_dunn(drug_syntox_df, val_col='synergy_zip', group_col='toxicity_category', p_adjust='bonferroni')
print('Dunn post-hoc test for ZIP synergy:')
print(dunn_zip)

zip_major_samples = major_zip.values.tolist()
zip_moderate_samples = moderate_zip.values.tolist()
zip_minor_samples = minor_zip.values.tolist()
zip_increasing_tox_samples = [zip_minor_samples, zip_moderate_samples, zip_major_samples]
jt_incr_zip = jonckheere_terpestra_test(zip_increasing_tox_samples)
print("Increasing toxicity for ZIP: ", jt_incr_zip )
zip_decreasing_tox_samples = [ zip_major_samples, zip_moderate_samples, zip_minor_samples]
jt_decr_zip = jonckheere_terpestra_test(zip_decreasing_tox_samples)
print("Decreasing toxicity for ZIP: ", jt_decr_zip)

# Run an ANOVA on the ZIP synergy scores for Major, Moderate, and Minor toxicity categoreis
f_statistic_anova_zip_tox, p_value_anova_zip_tox = stats.f_oneway(major_zip, moderate_zip, minor_zip)
print(f'ANOVA F-statistic for ZIP: {f_statistic_anova_zip_tox}')
print(f'P-value for ZIP: {p_value_anova_zip_tox}')

# Run a T test on ZIP between Major/Minor, Major/Moderate, and Moderate/Minor toxicity categories
major_minor_zip = stats.ttest_ind(major_zip, minor_zip)
major_moderate_zip = stats.ttest_ind(major_zip, moderate_zip)
moderate_minor_zip = stats.ttest_ind(moderate_zip, minor_zip)

# Run bonferroni correction on the p-values
ttest_p_values_zip = [major_minor_zip[1], major_moderate_zip[1], moderate_minor_zip[1]]
ttest_p_values_corrected_zip = multipletests(ttest_p_values_zip, method='bonferroni')
print(f'Major/Minor T-test: {major_minor_zip[1]} Corrected: {ttest_p_values_corrected_zip[1][0]}')
print(f'Major/Moderate T-test: {major_moderate_zip[1]} Corrected: {ttest_p_values_corrected_zip[1][1]}')
print(f'Moderate/Minor T-test: {moderate_minor_zip[1]} Corrected: {ttest_p_values_corrected_zip[1][2]}')

ax = sns.violinplot(data=drug_syntox_df, x='toxicity_category', y='synergy_zip', palette=color_dict, hue='toxicity_category', order=tox_order)
ax.set_xlabel('')
ax.set_ylabel('ZIP Synergy Score')
plt.legend(handles=legend_elements, bbox_to_anchor=(0.5,-0.2), loc='center', ncol=3)
plt.tight_layout()
plt.savefig('results/figure2/zip_v_toxcats_drugbank.png', dpi=700)
plt.close()

ax = sns.stripplot(data=drug_syntox_df, x='toxicity_category', y='synergy_zip', palette=color_dict, hue='toxicity_category', order=tox_order)
ax.set_xlabel('')
ax.set_ylabel('ZIP Synergy Score')
plt.legend(handles=legend_elements, bbox_to_anchor=(0.5,-0.2), loc='center', ncol=3)
plt.tight_layout()
plt.savefig('results/figure2/zip_v_toxcats_drugbank_strip.png', dpi=700)
plt.tight_layout()
plt.close()

#Loewe
major_loewe = drug_syntox_df[drug_syntox_df['toxicity_category'] == 'Major']['synergy_loewe']
moderate_loewe = drug_syntox_df[drug_syntox_df['toxicity_category'] == 'Moderate']['synergy_loewe']
minor_loewe = drug_syntox_df[drug_syntox_df['toxicity_category'] == 'Minor']['synergy_loewe']

h_statistic_loewe, p_value_loewe = stats.kruskal(major_loewe, moderate_loewe, minor_loewe)
print(f'Kruskal-Wallis H statistic for Loewe synergy: {h_statistic_loewe}')
print(f'P-value for Loewe synergy: {p_value_loewe}')

dunn_loewe = sp.posthoc_dunn(drug_syntox_df, val_col='synergy_loewe', group_col='toxicity_category', p_adjust='bonferroni')
print('Dunn post-hoc test for Loewe synergy:')
print(dunn_loewe)

loewe_major_samples = major_loewe.values.tolist()
loewe_moderate_samples = moderate_loewe.values.tolist()
loewe_minor_samples = minor_loewe.values.tolist()
loewe_increasing_tox_samples = [loewe_minor_samples, loewe_moderate_samples, loewe_major_samples]
jt_incr_loewe = jonckheere_terpestra_test(loewe_increasing_tox_samples)
print("Increasing toxicity for Loewe: ", jt_incr_loewe )
loewe_decreasing_tox_samples = [ loewe_major_samples, loewe_moderate_samples, loewe_minor_samples]
jt_decr_loewe = jonckheere_terpestra_test(loewe_decreasing_tox_samples)
print("Decreasing toxicity for Loewe: ", jt_decr_loewe)

# Run an ANOVA on the Loewe synergy scores for Major, Moderate, and Minor toxicity categoreis
f_statistic_anova_loewe_tox, p_value_anova_loewe_tox = stats.f_oneway(major_loewe, moderate_loewe, minor_loewe)
print(f'ANOVA F-statistic for Loewe: {f_statistic_anova_loewe_tox}')
print(f'P-value for Loewe: {p_value_anova_loewe_tox}')

# Run a T test on Loewe between Major/Minor, Major/Moderate, and Moderate/Minor toxicity categories
major_minor_loewe = stats.ttest_ind(major_loewe, minor_loewe)
major_moderate_loewe = stats.ttest_ind(major_loewe, moderate_loewe)
moderate_minor_loewe = stats.ttest_ind(moderate_loewe, minor_loewe)

# Run bonferroni correction on the p-values
ttest_p_values_loewe = [major_minor_loewe[1], major_moderate_loewe[1], moderate_minor_loewe[1]]
ttest_p_values_corrected_loewe = multipletests(ttest_p_values_loewe, method='bonferroni')
print(f'Major/Minor T-test: {major_minor_loewe[1]} Corrected: {ttest_p_values_corrected_loewe[1][0]}')
print(f'Major/Moderate T-test: {major_moderate_loewe[1]} Corrected: {ttest_p_values_corrected_loewe[1][1]}')
print(f'Moderate/Minor T-test: {moderate_minor_loewe[1]} Corrected: {ttest_p_values_corrected_loewe[1][2]}')

ax = sns.violinplot(data=drug_syntox_df, x='toxicity_category', y='synergy_loewe', palette=color_dict, hue='toxicity_category', order=tox_order)
ax.set_xlabel('')
ax.set_ylabel('Loewe Synergy Score')
plt.legend(handles=legend_elements, bbox_to_anchor=(0.5,-0.2), loc='center', ncol=3)
plt.tight_layout()
plt.savefig('results/figure2/loewe_v_toxcats_drugbank.png', dpi=700)
plt.close()

ax = sns.stripplot(data=drug_syntox_df, x='toxicity_category', y='synergy_loewe', palette=color_dict, hue='toxicity_category', order=tox_order)
ax.set_xlabel('')
ax.set_ylabel('Loewe Synergy Score')
plt.legend(handles=legend_elements, bbox_to_anchor=(0.5,-0.2), loc='center', ncol=3)
plt.tight_layout()
plt.savefig('results/figure2/loewe_v_toxcats_drugbank_strip.png', dpi=700)
plt.tight_layout()
plt.close()

# Write results to a file
with open('results/synergy_score_tox_categ_distrib/kruskal_dunn_jonck_majmodmin_syntox_drugbank.tsv', 'w') as f:
    f.write('Synergy Score\tTest\tTest Statistic\tP-value\n')
    f.write(f'Bliss\tKruskal-Wallis\t{h_statistic_bliss}\t{p_value_bliss}\n')
    f.write(f'Bliss\tDunn post-hoc\tMajor/Minor\t{dunn_bliss.iloc[0, 1]}\n')
    f.write(f'Bliss\tDunn post-hoc\tMajor/Moderate\t{dunn_bliss.iloc[0, 2]}\n')
    f.write(f'Bliss\tDunn post-hoc\tModerate/Minor\t{dunn_bliss.iloc[1, 2]}\n')
    f.write(f'Bliss\tJonckheere-Terpstra Increasing toxicity\t{jt_incr_bliss[0]}\t{jt_incr_bliss[1]}\n')
    f.write(f'Bliss\tJonckheere-Terpstra Decreasing toxicity\t{jt_decr_bliss[0]}\t{jt_decr_bliss[1]}\n')
    f.write(f'Bliss\tANOVA\t{f_statistic_anova_bliss_tox}\t{p_value_anova_bliss_tox}\n')
    f.write(f'Bliss\tT-test (bonf cor)\tMajor/Minor\t{ttest_p_values_corrected_bliss[1][0]}\n')
    f.write(f'Bliss\tT-test (bonf cor)\tMajor/Moderate\t{ttest_p_values_corrected_bliss[1][1]}\n')
    f.write(f'Bliss\tT-test (bonf cor)\tModerate/Minor\t{ttest_p_values_corrected_bliss[1][2]}\n')
    
    f.write(f'HSA\tKruskal-Wallis\t{h_statistic_hsa}\t{p_value_hsa}\n')
    f.write(f'HSA\tDunn post-hoc\tMajor/Minor\t{dunn_hsa.iloc[0, 1]}\n')
    f.write(f'HSA\tDunn post-hoc\tMajor/Moderate\t{dunn_hsa.iloc[0, 2]}\n')
    f.write(f'HSA\tDunn post-hoc\tModerate/Minor\t{dunn_hsa.iloc[1, 2]}\n')
    f.write(f'HSA\tJonckheere-Terpstra Increasing toxicity\t{jt_incr_hsa[0]}\t{jt_incr_hsa[1]}\n')
    f.write(f'HSA\tJonckheere-Terpstra Decreasing toxicity\t{jt_decr_hsa[0]}\t{jt_decr_hsa[1]}\n')
    f.write(f'HSA\tANOVA\t{f_statistic_anova_hsa_tox}\t{p_value_anova_hsa_tox}\n')
    f.write(f'HSA\tT-test (bonf cor)\tMajor/Minor\t{ttest_p_values_corrected_hsa[1][0]}\n')
    f.write(f'HSA\tT-test (bonf cor)\tMajor/Moderate\t{ttest_p_values_corrected_hsa[1][1]}\n')
    f.write(f'HSA\tT-test (bonf cor)\tModerate/Minor\t{ttest_p_values_corrected_hsa[1][2]}\n')

    f.write(f'Loewe\tKruskal-Wallis\t{h_statistic_loewe}\t{p_value_loewe}\n')
    f.write(f'Loewe\tDunn post-hoc\tMajor/Minor\t{dunn_loewe.iloc[0, 1]}\n')
    f.write(f'Loewe\tDunn post-hoc\tMajor/Moderate\t{dunn_loewe.iloc[0, 2]}\n')
    f.write(f'Loewe\tDunn post-hoc\tModerate/Minor\t{dunn_loewe.iloc[1, 2]}\n')
    f.write(f'Loewe\tJonckheere-Terpstra Increasing toxicity\t{jt_incr_loewe[0]}\t{jt_incr_loewe[1]}\n')
    f.write(f'Loewe\tJonckheere-Terpstra Decreasing toxicity\t{jt_decr_loewe[0]}\t{jt_decr_loewe[1]}\n')
    f.write(f'Loewe\tANOVA\t{f_statistic_anova_loewe_tox}\t{p_value_anova_loewe_tox}\n')
    f.write(f'Loewe\tT-test (bonf cor)\tMajor/Minor\t{ttest_p_values_corrected_loewe[1][0]}\n')
    f.write(f'Loewe\tT-test (bonf cor)\tMajor/Moderate\t{ttest_p_values_corrected_loewe[1][1]}\n')
    f.write(f'Loewe\tT-test (bonf cor)\tModerate/Minor\t{ttest_p_values_corrected_loewe[1][2]}\n')
    
    f.write(f'ZIP\tKruskal-Wallis\t{h_statistic_zip}\t{p_value_zip}\n')
    f.write(f'ZIP\tDunn post-hoc\tMajor/Minor\t{dunn_zip.iloc[0, 1]}\n')
    f.write(f'ZIP\tDunn post-hoc\tMajor/Moderate\t{dunn_zip.iloc[0, 2]}\n')
    f.write(f'ZIP\tDunn post-hoc\tModerate/Minor\t{dunn_zip.iloc[1, 2]}\n')
    f.write(f'ZIP\tJonckheere-Terpstra Increasing toxicity\t{jt_incr_zip[0]}\t{jt_incr_zip[1]}\n')
    f.write(f'ZIP\tJonckheere-Terpstra Decreasing toxicity\t{jt_decr_zip[0]}\t{jt_decr_zip[1]}\n')
    f.write(f'ZIP\tANOVA\t{f_statistic_anova_zip_tox}\t{p_value_anova_zip_tox}\n')
    f.write(f'ZIP\tT-test (bonf cor)\tMajor/Minor\t{ttest_p_values_corrected_zip[1][0]}\n')
    f.write(f'ZIP\tT-test (bonf cor)\tMajor/Moderate\t{ttest_p_values_corrected_zip[1][1]}\n')
    f.write(f'ZIP\tT-test (bonf cor)\tModerate/Minor\t{ttest_p_values_corrected_zip[1][2]}\n')


Kruskal-Wallis H statistic for Bliss synergy: 974.1294260454137
P-value for Bliss synergy: 2.9544919053525385e-212
Dunn post-hoc test for Bliss synergy:
                  Major          Minor       Moderate
Major      1.000000e+00  2.046611e-212   1.006816e-16
Minor     2.046611e-212   1.000000e+00  2.154424e-116
Moderate   1.006816e-16  2.154424e-116   1.000000e+00
Increasing toxicity for Bliss:  (25.08656782068397, 0.0)
Decreasing toxicity for Bliss:  (-25.08656782068397, 1.0)
ANOVA F-statistic for bliss: 638.8604957319966
P-value for bliss: 3.4398376034526776e-276
Major/Minor T-test: 1.2274561664052582e-294 Corrected: 3.682368499215775e-294
Major/Moderate T-test: 1.7829774879475262e-44 Corrected: 5.348932463842579e-44
Moderate/Minor T-test: 2.812422925939104e-94 Corrected: 8.437268777817313e-94
Kruskal-Wallis H statistic for HSA synergy: 244.20576597303574
P-value for HSA synergy: 9.36249706314586e-54
Dunn post-hoc test for HSA synergy:
                 Major         Minor      Mode

DEPRECATED combining moderate and minor toxicity not really of interest

In [ ]:
# # What if you combine moderate and minor toxicity categories?
# drug_syntox_df['toxicity_category'] = drug_syntox_df['toxicity_category'].replace('Moderate', 'Moderate/Minor')
# drug_syntox_df['toxicity_category'] = drug_syntox_df['toxicity_category'].replace('Minor', 'Moderate/Minor')

# # BLISS
# moderate_minor_bliss = drug_syntox_df[drug_syntox_df['toxicity_category'] == 'Moderate/Minor']['synergy_bliss']

# h_statistic_bliss_tolerable, p_value_bliss_tolerable = stats.kruskal(major_bliss, moderate_minor_bliss)
# print(f'Kruskal-Wallis H statistic for Bliss synergy (Moderate/Minor combined): {h_statistic_bliss_tolerable}')
# print(f'P-value for Bliss synergy (Moderate/Minor combined): {p_value_bliss_tolerable}')

# mwy_statistic_bliss_tolerable, mwy_p_value_bliss_tolerable = stats.mannwhitneyu(major_bliss, moderate_minor_bliss, alternative='two-sided')
# print(f'Mann-Whitney U statistic for Bliss synergy (Major vs. Moderate/Minor): {mwy_statistic_bliss_tolerable}')
# print(f'P-value for Bliss synergy (Major vs. Moderate/Minor): {mwy_p_value_bliss_tolerable}')


# # HSA
# moderate_minor_hsa = drug_syntox_df[drug_syntox_df['toxicity_category'] == 'Moderate/Minor']['synergy_hsa']

# h_statistic_hsa_tolerable, p_value_hsa_tolerable = stats.kruskal(major_hsa, moderate_minor_hsa)
# print(f'Kruskal-Wallis H statistic for HSA synergy (Moderate/Minor combined): {h_statistic_hsa_tolerable}')
# print(f'P-value for HSA synergy (Moderate/Minor combined): {p_value_hsa_tolerable}')

# mwy_statistic_hsa_tolerable, mwy_p_value_hsa_tolerable = stats.mannwhitneyu(major_hsa, moderate_minor_hsa, alternative='two-sided')
# print(f'Mann-Whitney U statistic for HSA synergy (Major vs. Moderate/Minor): {mwy_statistic_hsa_tolerable}')
# print(f'P-value for HSA synergy (Major vs. Moderate/Minor): {mwy_p_value_hsa_tolerable}')

# # ZIP
# moderate_minor_zip = drug_syntox_df[drug_syntox_df['toxicity_category'] == 'Moderate/Minor']['synergy_zip']

# h_statistic_zip_tolerable, p_value_zip_tolerable = stats.kruskal(major_zip, moderate_minor_zip)
# print(f'Kruskal-Wallis H statistic for ZIP synergy (Moderate/Minor combined): {h_statistic_zip_tolerable}')
# print(f'P-value for ZIP synergy (Moderate/Minor combined): {p_value_zip_tolerable}')

# mwy_statistic_zip_tolerable, mwy_p_value_zip_tolerable = stats.mannwhitneyu(major_zip, moderate_minor_zip, alternative='two-sided')
# print(f'Mann-Whitney U statistic for ZIP synergy (Major vs. Moderate/Minor): {mwy_statistic_zip_tolerable}')
# print(f'P-value for ZIP synergy (Major vs. Moderate/Minor): {mwy_p_value_zip_tolerable}')

# # LOEWE
# moderate_minor_loewe = drug_syntox_df[drug_syntox_df['toxicity_category'] == 'Moderate/Minor']['synergy_loewe']

# h_statistic_loewe_tolerable, p_value_loewe_tolerable = stats.kruskal(major_loewe, moderate_minor_loewe)
# print(f'Kruskal-Wallis H statistic for Loewe synergy (Moderate/Minor combined): {h_statistic_loewe_tolerable}')
# print(f'P-value for Loewe synergy (Moderate/Minor combined): {p_value_loewe_tolerable}')

# mwy_statistic_loewe_tolerable, mwy_p_value_loewe_tolerable = stats.mannwhitneyu(major_loewe, moderate_minor_loewe, alternative='two-sided')
# print(f'Mann-Whitney U statistic for Loewe synergy (Major vs. Moderate/Minor): {mwy_statistic_loewe_tolerable}')
# print(f'P-value for Loewe synergy (Major vs. Moderate/Minor): {mwy_p_value_loewe_tolerable}')

# with open('results/syntox_relationship_analysis/kruskal_mannwhitney_tolerable_syntox.tsv', 'w') as f:
#     f.write('Synergy Score\tTest\tTest Statistic\tP-value\n')
#     f.write(f'Bliss\tKruskal-Wallis\t{h_statistic_bliss_tolerable}\t{p_value_bliss_tolerable}\n')
#     f.write(f'Bliss\tMann-Whitney U\t{mwy_statistic_bliss_tolerable}\t{mwy_p_value_bliss_tolerable}\n')
#     f.write(f'HSA\tKruskal-Wallis\t{h_statistic_hsa_tolerable}\t{p_value_hsa_tolerable}\n')
#     f.write(f'HSA\tMann-Whitney U\t{mwy_statistic_hsa_tolerable}\t{mwy_p_value_hsa_tolerable}\n')
#     f.write(f'Loewe\tKruskal-Wallis\t{h_statistic_loewe_tolerable}\t{p_value_loewe_tolerable}\n')
#     f.write(f'Loewe\tMann-Whitney U\t{mwy_statistic_loewe_tolerable}\t{mwy_p_value_loewe_tolerable}\n')
#     f.write(f'ZIP\tKruskal-Wallis\t{h_statistic_zip_tolerable}\t{p_value_zip_tolerable}\n')
#     f.write(f'ZIP\tMann-Whitney U\t{mwy_statistic_zip_tolerable}\t{mwy_p_value_zip_tolerable}\n')


In [ ]:
# # Generate histograms for tolerable toxicity category (Moderate/Minor combined)
# drug_syntox_df[drug_syntox_df['toxicity_category'] == 'Moderate/Minor']['synergy_bliss'].hist(bins=100)
# plt.xlabel('Synergy Bliss')
# plt.ylabel('Frequency')
# plt.title('Synergy Bliss Distribution for Moderate/Minor Toxicity')
# plt.savefig('results/synergy_score_tox_categ_distrib/bliss_tolerable_drugbank.png')
# plt.close()

# drug_syntox_df[drug_syntox_df['toxicity_category'] == 'Moderate/Minor']['synergy_hsa'].hist(bins=100)
# plt.xlabel('Synergy HSA')
# plt.ylabel('Frequency')
# plt.title('Synergy HSA Distribution for Moderate/Minor Toxicity')
# plt.savefig('results/synergy_score_tox_categ_distrib/hsa_tolerable_drugbank.png')
# plt.close()

# drug_syntox_df[drug_syntox_df['toxicity_category'] == 'Moderate/Minor']['synergy_zip'].hist(bins=100)
# plt.xlabel('Synergy ZIP')
# plt.ylabel('Frequency')
# plt.title('Synergy ZIP Distribution for Moderate/Minor Toxicity')
# plt.savefig('results/synergy_score_tox_categ_distrib/zip_tolerable_drugbank.png')
# plt.close()

# drug_syntox_df[drug_syntox_df['toxicity_category'] == 'Moderate/Minor']['synergy_loewe'].hist(bins=100)
# plt.xlabel('Synergy Loewe')
# plt.ylabel('Frequency')
# plt.title('Synergy Loewe Distribution for Moderate/Minor Toxicity')
# plt.savefig('results/synergy_score_tox_categ_distrib/loewe_tolerable_drugbank.png')
# plt.close()

Plot the relationship between synergy scores, how do they relate to one another?

In [6]:
# Plot scatter plot of synergy_bliss vs. synergy_hsa
drug_syntox_df.plot.scatter(x='synergy_bliss', y='synergy_hsa')
plt.xlabel('Synergy Bliss')
plt.ylabel('Synergy HSA')
plt.title('Synergy HSA vs. Synergy Bliss')
plt.savefig('results/synergy_score_scatter_plots/hsa_v_bliss_drugbank.png')
plt.close()

# Plot scatter plot of synergy_bliss vs. synergy_zip
drug_syntox_df.plot.scatter(x='synergy_bliss', y='synergy_zip')
plt.xlabel('Synergy Bliss')
plt.ylabel('Synergy ZIP')
plt.title('Synergy ZIP vs. Synergy Bliss')
plt.savefig('results/synergy_score_scatter_plots/zip_v_bliss_drugbank.png')
plt.close()

# Plot scatter plot of synergy_bliss vs. synergy_loewe
drug_syntox_df.plot.scatter(x='synergy_bliss', y='synergy_loewe')
plt.xlabel('Synergy Bliss')
plt.ylabel('Synergy Loewe')
plt.title('Synergy Loewe vs. Synergy Bliss')
plt.savefig('results/synergy_score_scatter_plots/loewe_v_bliss_drugbank.png')
plt.close()

# Plot scatter plot of synergy_hsa vs. synergy_zip
drug_syntox_df.plot.scatter(x='synergy_hsa', y='synergy_zip')
plt.xlabel('Synergy HSA')
plt.ylabel('Synergy ZIP')
plt.title('Synergy ZIP vs. Synergy HSA')
plt.savefig('results/synergy_score_scatter_plots/zip_v_hsa_drugbank.png')
plt.close()

# Plot scatter plot of synergy_hsa vs. synergy_loewe
drug_syntox_df.plot.scatter(x='synergy_hsa', y='synergy_loewe')
plt.xlabel('Synergy HSA')
plt.ylabel('Synergy Loewe')
plt.title('Synergy Loewe vs. Synergy HSA')
plt.savefig('results/synergy_score_scatter_plots/loewe_v_hsa_drugbank.png')
plt.close()

# Plot scatter plot of synergy_zip vs. synergy_loewe
drug_syntox_df.plot.scatter(x='synergy_zip', y='synergy_loewe')
plt.xlabel('Synergy ZIP')
plt.ylabel('Synergy Loewe')
plt.title('Synergy Loewe vs. Synergy ZIP')
plt.savefig('results/synergy_score_scatter_plots/loewe_v_zip_drugbank.png')
plt.close()
